# Sentinel-3 Product Dissemination Unit Comparison

    Version: 2.0
    Date:    10/04/2019
    Author:  Ben Loveday and Hayley Evers-King (Plymouth Marine Laboratory)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

This code shows you the spatial extent of Sentinel-3 Level-1B and Level-2 NRT and NTC products from three different sensors (OLCI, SLSTR and SRAL). To run this script, you will need to point it to where your data is. To help to find your data, please complete the MYPATH variable below with the output generated by the /Configuration_Testing/Data_Path_Checker.ipynb Jupyter notebook in the Configuration_Testing folder.

In [ ]:
import os
# e.g. MYPATH = os.path.join("C:/","Users","me","Desktop","Training_course_data")
MYPATH = "<please insert your path from Data_Path_Checker.ipynb here, removing the quotes and chevrons>"

In [ ]:
%matplotlib inline

# libraries are imported here, and we can import any library with an alias that allows us easy access to them later.
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import warnings
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
warnings.filterwarnings('ignore')

In [ ]:
land_resolution = '50m'
land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                        edgecolor='k',
                                        facecolor=cfeature.COLORS['land'])

Sentinel-3 products from each sensor are delivered with different product dissemination units with different coverage. The files below will be used to make a plot that show all of the coverage options for each sensor and product, for a given timeliness.

In [ ]:
input_root = os.path.join(MYPATH,'S3_test_data')

# OLCI REDUCED RESOLUTION L2 FILE
OLCI_RR_CRD_FILE = "S3A_OL_2_WRR____20181026T082318_20181026T090734_20181027T135931_2656_037_178______MAR_O_NT_002.SEN3/geo_coordinates.nc"

# OLCI FULL RESOLUTION L2 FILE
OLCI_FR_CRD_FILE = "S3A_OL_2_WFR____20181026T083119_20181026T083419_20181027T153801_0179_037_178_2340_MAR_O_NT_002.SEN3/geo_coordinates.nc"

# SLSTR NADIR L1 FILE
SLSTR_L1_CRD_N_FILE = "S3A_SL_1_RBT____20181026T083119_20181026T083419_20181027T170641_0179_037_178_2340_MAR_O_NT_003.SEN3/geodetic_in.nc"

# SLSTR OBLIQUE L1 FILE
SLSTR_L1_CRD_O_FILE = "S3A_SL_1_RBT____20181026T083119_20181026T083419_20181027T170641_0179_037_178_2340_MAR_O_NT_003.SEN3/geodetic_io.nc"

# SLSTR L2 FILE (NADIR AND DUAL)
SLSTR_L2_FILE = "S3A_SL_2_WST____20181026T090801_20181026T104900_20181027T192105_6059_037_178______MAR_O_NT_003.SEN3/20181026090801-MAR-L2P_GHRSST-SSTskin-SLSTRA-20181027192105-v02.0-fv01.0_subset.nc"

# SRAL L2 FILE
SRAL_L2_FILE = "S3A_SR_2_WAT____20181026T081732_20181026T090331_20181028T000914_2759_037_178______MAR_O_ST_003.SEN3/standard_measurement.nc"

Lets load in the data...

In [ ]:
# OLCI REDUCED RESOLUTION --------------------------
ds1 = xr.open_dataset(os.path.join(input_root,OLCI_RR_CRD_FILE))
OLCI_RR_LAT = ds1.latitude
OLCI_RR_LON = ds1.longitude
ds1.close()

# OLCI FULL RESOLUTION --------------------------
ds1 = xr.open_dataset(os.path.join(input_root,OLCI_FR_CRD_FILE))
OLCI_FR_LAT = ds1.latitude
OLCI_FR_LON = ds1.longitude
ds1.close()

# SLSTR NADIR L1  --------------------------
ds1 = xr.open_dataset(os.path.join(input_root,SLSTR_L1_CRD_N_FILE))
SLSTR_L1_LAT_N = ds1.latitude_in
SLSTR_L1_LON_N = ds1.longitude_in
ds1.close()

# SLSTR OBLIQUE L1  --------------------------
ds1 = xr.open_dataset(os.path.join(input_root,SLSTR_L1_CRD_O_FILE))
SLSTR_L1_LAT_O = ds1.latitude_io
SLSTR_L1_LON_O = ds1.longitude_io
ds1.close()

# SLSTR L2  --------------------------
ds1 = xr.open_dataset(os.path.join(input_root,SLSTR_L2_FILE))
SLSTR_L2_LAT = ds1.lat
SLSTR_L2_LON = ds1.lon
SLSTR_L2_ALG = np.squeeze(ds1.sst_algorithm_type)
SLSTR_L2_LON_D = SLSTR_L2_LON.copy().values
SLSTR_L2_LON_D[SLSTR_L2_ALG.values<3] = np.nan
ds1.close()

# SRAL L2  --------------------------
ds1 = xr.open_dataset(os.path.join(input_root,SRAL_L2_FILE))
SRAL_L2_LAT = ds1.lat_20_ku
SRAL_L2_LON = ds1.lon_20_ku
ds1.close()

SLSTR data is full orbit, which is hard to plot. We will manipulate it slightly near the date line to make our plots better.

In [ ]:
# remove international date line edges to stop bad interpolation
SLSTR_L2_COL = np.ones(np.shape(SLSTR_L2_LON))
SLSTR_L2_COL_D = np.ones(np.shape(SLSTR_L2_LON_D))
SLSTR_L2_COL[SLSTR_L2_LON < -179.0] = np.nan
SLSTR_L2_COL[SLSTR_L2_LON > 179.0] = np.nan
SLSTR_L2_COL_D[SLSTR_L2_LON_D < -179.0] = np.nan
SLSTR_L2_COL_D[SLSTR_L2_LON_D > 179.0] = np.nan

Now we plot the data. This may take a little bit of time...

In [ ]:
fig1 = plt.figure(figsize=(20, 40), dpi=300)

gs  = gridspec.GridSpec(2, 1)

#---
# plot dumps/stripes
m = plt.subplot(gs[0,0], projection = ccrs.PlateCarree(central_longitude=0.0))
levels = [0.0, 1.0]
f1 = m.contourf(OLCI_RR_LON, OLCI_RR_LAT, OLCI_RR_LAT*0.0+1.0, levels, colors=('#A2CD5A'), zorder = 1, alpha=0.75)
f2 = m.plot(SRAL_L2_LON, SRAL_L2_LAT, marker='o', linewidth=0.0, color='b', zorder=4, markersize=2)
f3 = m.contourf(SLSTR_L2_LON, SLSTR_L2_LAT, SLSTR_L2_COL, levels, colors=('#FFB90F'), zorder = 2, alpha = 0.75)

m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}
plt.title('Sentinel-3 Product coverage: dumps', fontsize=20)

patch1 = mpatches.Patch(facecolor='#A2CD5A', edgecolor='k', label='OLCI RR L1 & L2 (NRT & NTC)')
patch2 = mpatches.Patch(facecolor='#FFB90F', edgecolor='k', label='SLSTR NADIR L2 NTC')
patch3 = mpatches.Patch(facecolor='b',       edgecolor='k', label='SRAL L1 & L2 (NRT & NTC)')
plt.legend(loc=9, bbox_to_anchor=(0.5, -0.1), handles=[patch1, patch2, patch3], fontsize=16)

# ---
# plot frames/granules
m = plt.subplot(gs[1,0], projection=ccrs.PlateCarree(central_longitude=0.0))
levels = [0.0, 1.0]
f1 = m.contourf(OLCI_FR_LON, OLCI_FR_LAT, OLCI_FR_LAT*0.0+1.0, levels, colors=('#556B2F'), zorder = 2,\
                alpha=1.0, hatches=['/'])
f2 = m.contourf(SLSTR_L1_LON_O, SLSTR_L1_LAT_O, SLSTR_L1_LAT_O*0.0+1.0, levels, colors=('#FFB90F'), zorder = 3, alpha=0.8)
f3 = m.contourf(SLSTR_L1_LON_N, SLSTR_L1_LAT_N, SLSTR_L1_LAT_N*0.0+1.0, levels, colors=('b'),\
                zorder = 1, alpha=1.0)

m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}
plt.title('Sentinel-3 Product coverage: frames', fontsize=20)
patch1 = mpatches.Patch(facecolor='b',       edgecolor='k', label='SLSTR NADIR L1 (NRT & NTC) and L2 (NRT)')
patch2 = mpatches.Patch(facecolor='#A2CD5A', edgecolor='k', label='OLCI FR L1 & L2 (NRT & NTC)', hatch='///')
patch3 = mpatches.Patch(facecolor='#FFB90F', edgecolor='k', label='SLSTR  L1 DUAL/OBLIQUE (NRT & NTC) and L2 (NRT)')
plt.legend(loc=9, bbox_to_anchor=(0.5, -0.1), handles=[patch1, patch2, patch3], fontsize=16)
plt.show()